Daily Challenge: W8_D5

Building an Agent with LangGraph and Gemini API

---

## What You'll Learn

- How to create a stateful application using **LangGraph**.  
- How to integrate **Gemini API** (via LangChain) into your LangGraph application.  
- How to define and manipulate **state** using `TypedDict` and node functions.  
- How to simulate dynamic, tool-augmented behavior (like menus and ordering) in a conversational loop.  
- How to model **conditional transitions**, loops, and user interaction using LangGraph.

---

## What You Will Create

A conversational **cafe ordering system** called **BaristaBot**. This bot will:

- Use natural language to take coffee/tea orders.  
- Offer a real-time menu using a tool.  
- Confirm and modify orders.  
- Loop through conversation until an order is placed.  
- Handle tool calls using LangGraph's `ToolNode` mechanism.  

This graph-based app simulates a **real-world cafe assistant** using AI + state management.

---

## Task Overview

In this notebook, you will use **LangGraph** to define a **stateful graph-based application** built on top of the **Gemini API**.

You will build a simulated cafe ordering system, **BaristaBot**. It will:

- Provide a looping chat interface to customers where they can order cafe beverages using natural language.  
- Use nodes to represent the cafe's **live menu** and the **back room** ordering system.  

BaristaBot is used in other Gemini API demos. If you prefer a more minimal implementation, check out the **BaristaBot function calling example** that implements a similar system using only the Gemini API Python SDK and function calling.

---

## Steps

1. **Install and import** the LangGraph SDK and LangChain support for Gemini API.  
2. **Set up your API key** (via Kaggle Secrets or manual environment variable).  
3. **Define the application state and system instructions** (conversation rules + order tracking).  
4. **Create a single-turn chatbot node** and connect it in a basic graph.  
5. **Run and visualize the graph** to test the first interaction.  
6. **Manually add a second user interaction** to simulate a continued conversation.  
7. **Add a human node** for automated conversation looping (user ↔ chatbot).  
8. **Introduce conditional transitions** (exit when user types `q` or finishes order).  
9. **Add a live menu tool** using `@tool` and integrate it with the chatbot.  
10. **Handle orders with stateful tools** (add to order, clear order, confirm order, place order).  
11. **Combine all nodes** into the final graph (chatbot + human + menu + ordering).  
12. **Run the full BaristaBot application** and test ordering flow.

---

## Goal

By the end of this notebook, you will have a **fully functional conversational agent** that simulates a cafe ordering system with:

- State management via LangGraph  
- Tool integration for menu and orders  
- Gemini LLM powering the chatbot logic  

## Step 1: Installation and Imports

In [3]:
# Install required packages
%pip install -qU 'langgraph==0.2.45' 'langchain-google-genai==2.0.4'

# Import necessary libraries
import os
from typing import Annotated, Literal
from typing_extensions import TypedDict
from collections.abc import Iterable
from random import randint
from pprint import pprint

# LangGraph and LangChain imports
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages.ai import AIMessage
from langchain_core.messages.tool import ToolMessage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 497.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 2.9 MB/s eta 0:00:00


We start by installing the required packages and importing all necessary libraries for building our LangGraph application with Gemini API integration.

## Step 2: API Key Setup

In [2]:
import os
import getpass
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Entre ta clé API: ")

Entre ta clé API: ··········


In [5]:
# Test 2 : Vérifier que la connexion à Gemini fonctionne
try:
    from langchain_google_genai import ChatGoogleGenerativeAI

    print("🔄 Test de connexion à Gemini...")

    # Crée une instance du modèle
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

    # Test simple
    response = llm.invoke("Dis juste 'Hello' en réponse")

    print("✅ Connexion réussie !")
    print(f"Réponse de Gemini : {response.content}")

except Exception as e:
    print("❌ Erreur de connexion :")
    print(f"Détail : {e}")

    # Messages d'aide selon l'erreur
    if "API_KEY" in str(e):
        print("💡 Problème de clé API - vérifies qu'elle est bien configurée")
    elif "quota" in str(e).lower():
        print("💡 Problème de quota - tu as peut-être dépassé la limite gratuite")
    elif "permission" in str(e).lower():
        print("💡 Problème de permissions - vérifies que l'API Gemini est activée")

🔄 Test de connexion à Gemini...
✅ Connexion réussie !
Réponse de Gemini : Hello



## Step 3: Define Core State and Instructions

In [6]:
# Define the application state schema
class OrderState(TypedDict):
    """State representing the customer's order conversation."""

    # Chat conversation history with automatic message appending
    messages: Annotated[list, add_messages]

    # Customer's current order as list of strings
    order: list[str]

    # Flag to indicate if ordering process is complete
    finished: bool

# System instruction defining chatbot behavior and rules
BARISTABOT_SYSINT = (
    "system",  # Message type indicator
    "You are a BaristaBot, an interactive cafe ordering system. A human will talk to you about the "
    "available products you have and you will answer any questions about menu items (and only about "
    "menu items - no off-topic discussion, but you can chat about the products and their history). "
    "The customer will place an order for 1 or more items from the menu, which you will structure "
    "and send to the ordering system after confirming the order with the human. "
    "\n\n"
    "Add items to the customer's order with add_to_order, and reset the order with clear_order. "
    "To see the contents of the order so far, call get_order (this is shown to you, not the user) "
    "Always confirm_order with the user (double-check) before calling place_order. Calling confirm_order will "
    "display the order items to the user and returns their response to seeing the list. Their response may contain modifications. "
    "Always verify and respond with drink and modifier names from the MENU before adding them to the order. "
    "If you are unsure a drink or modifier matches those on the MENU, ask a question to clarify or redirect. "
    "You only have the modifiers listed on the menu. "
    "Once the customer has finished ordering items, Call confirm_order to ensure it is correct then make "
    "any necessary updates and then call place_order. Once place_order has returned, thank the user and "
    "say goodbye!",
)

# Welcome message to start conversations
WELCOME_MSG = "Welcome to the BaristaBot cafe. Type `q` to quit. How may I serve you today?"

print("✅ State schema and system instructions defined")

✅ State schema and system instructions defined


We define the state structure using TypedDict to track conversation messages, current order, and completion status. The system instruction establishes the chatbot's role and operational rules.

## Step 4: Simple Chatbot Node

In [7]:
# Initialize Gemini model with fast flash variant
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

def chatbot(state: OrderState) -> OrderState:
    """Simple chatbot node that processes messages and returns AI response."""
    # Combine system instruction with conversation history
    message_history = [BARISTABOT_SYSINT] + state["messages"]
    # Get model response and return updated state
    return {"messages": [llm.invoke(message_history)]}

# Create initial graph builder with state schema
graph_builder = StateGraph(OrderState)

# Add chatbot as a node in the graph
graph_builder.add_node("chatbot", chatbot)

# Define entry point from START to chatbot
graph_builder.add_edge(START, "chatbot")

# Compile the graph for execution
chat_graph = graph_builder.compile()

print("✅ Simple chatbot graph created")

✅ Simple chatbot graph created


This creates a basic single-node graph with a chatbot that can process one conversational turn using the Gemini model.

## Step 5: Visualize Initial Graph

In [8]:
try:
    # Display graph structure as visual diagram
    from IPython.display import Image
    Image(chat_graph.get_graph().draw_mermaid_png())
except Exception as e:
    print("Graph visualization not available in this environment")
    print("Graph structure: START -> chatbot -> END")

We attempt to visualize the graph structure to understand the flow of our application.

## Step 6: Test Simple Chatbot

In [9]:
# Test single conversation turn
user_msg = "What drinks do you have?"
state = chat_graph.invoke({"messages": [("user", user_msg)], "order": [], "finished": False})

# Display conversation messages
print("=== Conversation ===")
for msg in state["messages"]:
    print(f"{type(msg).__name__}: {msg.content}")

=== Conversation ===
HumanMessage: What drinks do you have?
AIMessage: We have a variety of coffee drinks, including espresso, Americano, macchiato, latte, cappuccino, mocha, and iced coffee.  We also offer tea, hot chocolate, and juices.



We test the basic chatbot functionality with a simple user query about available drinks.

## Step 7: Add Second Conversation Turn

In [10]:
# Continue conversation with another turn
user_msg = "Tell me about your coffee options"

# Append new user message to existing state
state["messages"].append(("user", user_msg))
state = chat_graph.invoke(state)

print("\n=== Extended Conversation ===")
for msg in state["messages"]:
    print(f"{type(msg).__name__}: {msg.content}")


=== Extended Conversation ===
HumanMessage: What drinks do you have?
AIMessage: We have a variety of coffee drinks, including espresso, Americano, macchiato, latte, cappuccino, mocha, and iced coffee.  We also offer tea, hot chocolate, and juices.

HumanMessage: Tell me about your coffee options
AIMessage: Our espresso is made with freshly roasted, high-quality beans.  The Americano is espresso diluted with hot water. The macchiato is espresso marked with a dollop of foamed milk.  The latte is espresso with steamed milk and a thin layer of foam. The cappuccino has equal parts espresso, steamed milk, and foamed milk. The mocha is espresso with chocolate syrup, steamed milk, and whipped cream. Our iced coffee is brewed coffee served over ice.  Do any of those sound good to you?



This demonstrates how state is maintained across multiple conversation turns.

## Step 8: Add Human Interaction Node

In [11]:
def human_node(state: OrderState) -> OrderState:
    """Handle human interaction - display AI message and get user input."""
    # Get the last message from the AI
    last_msg = state["messages"][-1]
    print("Model:", last_msg.content)

    # Get user input from console
    user_input = input("User: ")

    # Check for quit commands and set finished flag
    if user_input in {"q", "quit", "exit", "goodbye"}:
        state["finished"] = True

    # Return updated state with new user message
    return state | {"messages": [("user", user_input)]}

def chatbot_with_welcome_msg(state: OrderState) -> OrderState:
    """Enhanced chatbot with welcome message handling."""

    if state["messages"]:
        # Continue existing conversation
        new_output = llm.invoke([BARISTABOT_SYSINT] + state["messages"])
    else:
        # Start new conversation with welcome
        new_output = AIMessage(content=WELCOME_MSG)

    return state | {"messages": [new_output]}

print("✅ Human interaction node created")

✅ Human interaction node created


We add a human node to handle user input and output display, plus enhance the chatbot to handle conversation initiation.

## Step 9: Add Conditional Routing

In [13]:
def maybe_exit_human_node(state: OrderState) -> Literal["chatbot", "__end__"]:
    """Route to chatbot or end based on finished flag."""
    if state.get("finished", False):
        return END  # Exit application
    else:
        return "chatbot"  # Continue conversation

# Build new graph with human interaction
graph_builder = StateGraph(OrderState)

# Add both chatbot and human nodes
graph_builder.add_node("chatbot", chatbot_with_welcome_msg)
graph_builder.add_node("human", human_node)

# Define flow: START -> chatbot -> human -> [chatbot|END]
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", "human")
graph_builder.add_conditional_edges("human", maybe_exit_human_node)  # ← CORRECTION ICI

# Compile interactive chat graph
chat_with_human_graph = graph_builder.compile()

print("✅ Interactive chat graph with conditional routing created")

✅ Interactive chat graph with conditional routing created


We implement conditional routing to allow the conversation to continue or end based on user input, creating a proper chat loop.

## Step 10: Add Menu Tool

In [14]:
@tool
def get_menu() -> str:
    """Provide the latest up-to-date cafe menu with all available items and modifiers."""
    return """
    MENU:
    Coffee Drinks:
    Espresso
    Americano
    Cold Brew

    Coffee Drinks with Milk:
    Latte
    Cappuccino
    Cortado
    Macchiato
    Mocha
    Flat White

    Tea Drinks:
    English Breakfast Tea
    Green Tea
    Earl Grey

    Tea Drinks with Milk:
    Chai Latte
    Matcha Latte
    London Fog

    Other Drinks:
    Steamer
    Hot Chocolate

    Modifiers:
    Milk options: Whole, 2%, Oat, Almond, 2% Lactose Free; Default option: whole
    Espresso shots: Single, Double, Triple, Quadruple; default: Double
    Caffeine: Decaf, Regular; default: Regular
    Hot-Iced: Hot, Iced; Default: Hot
    Sweeteners (option to add one or more): vanilla sweetener, hazelnut sweetener, caramel sauce, chocolate sauce, sugar free vanilla sweetener
    Special requests: any reasonable modification that does not involve items not on the menu, for example: 'extra hot', 'one pump', 'half caff', 'extra foam', etc.

    "dirty" means add a shot of espresso to a drink that doesn't usually have it, like "Dirty Chai Latte".
    "Regular milk" is the same as 'whole milk'.
    "Sweetened" means add some regular sugar, not a sweetener.

    Soy milk has run out of stock today, so soy is not available.
    """

# Create tool node and bind tools to model
tools = [get_menu]
tool_node = ToolNode(tools)
llm_with_tools = llm.bind_tools(tools)

print("✅ Menu tool created and bound to model")

✅ Menu tool created and bound to model


We create a tool that provides the current menu information, making it available to the chatbot for dynamic menu queries.

## Step 11: Enhanced Chatbot with Tools

In [16]:
def maybe_route_to_tools(state: OrderState) -> Literal["tools", "human"]:
    """Route between human or tool nodes based on tool calls."""
    if not (msgs := state.get("messages", [])):
        raise ValueError(f"No messages found when parsing state: {state}")

    # Check last message for tool calls
    msg = msgs[-1]

    # Route to tools if AI made tool calls, otherwise to human
    if hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
        return "tools"
    else:
        return "human"

def chatbot_with_tools(state: OrderState) -> OrderState:
    """Enhanced chatbot with tool calling capabilities."""
    # Set default values for state
    defaults = {"order": [], "finished": False}

    if state["messages"]:
        # Continue conversation with tool-enabled model
        new_output = llm_with_tools.invoke([BARISTABOT_SYSINT] + state["messages"])
    else:
        # Start with welcome message
        new_output = AIMessage(content=WELCOME_MSG)

    # Merge defaults with current state, updating only messages
    return defaults | state | {"messages": [new_output]}

# Build graph with tool support
graph_builder = StateGraph(OrderState)

# Add all nodes: chatbot, human, and tools
graph_builder.add_node("chatbot", chatbot_with_tools)
graph_builder.add_node("human", human_node)
graph_builder.add_node("tools", tool_node)

# Define routing logic - CORRECTIONS ICI
graph_builder.add_conditional_edges("chatbot", maybe_route_to_tools)  # ← avec S
graph_builder.add_conditional_edges("human", maybe_exit_human_node)   # ← avec S
graph_builder.add_edge("tools", "chatbot")  # Tools always return to chatbot
graph_builder.add_edge(START, "chatbot")

# Compile graph with menu tools
graph_with_menu = graph_builder.compile()

print("✅ Enhanced chatbot with menu tools created")

✅ Enhanced chatbot with menu tools created


We enhance the chatbot to support tool calling, allowing it to access the menu dynamically during conversations.

## Step 12: Order Management Tools

In [17]:
# Define order management tool schemas (implementation in separate node)
@tool
def add_to_order(drink: str, modifiers: Iterable[str]) -> str:
    """Add specified drink with modifiers to customer's order."""
    pass  # Implementation in order_node

@tool
def confirm_order() -> str:
    """Ask customer to confirm their current order."""
    pass  # Implementation in order_node

@tool
def get_order() -> str:
    """Return current order contents for internal use."""
    pass  # Implementation in order_node

@tool
def clear_order():
    """Remove all items from customer's order."""
    pass  # Implementation in order_node

@tool
def place_order() -> int:
    """Send order to kitchen and return estimated completion time."""
    pass  # Implementation in order_node

print("✅ Order management tool schemas defined")

✅ Order management tool schemas defined


We define the schemas for order management tools that will be implemented in a dedicated order handling node.

## Step 13: Order Processing Node

In [18]:
def order_node(state: OrderState) -> OrderState:
    """Handle all order-related tool calls and state modifications."""
    # Get the last tool message
    tool_msg = state["messages"][-1]
    order = state.get("order", []).copy()  # Work with copy to avoid mutation
    outbound_msgs = []
    order_placed = False

    # Process each tool call
    for tool_call in tool_msg.tool_calls:

        if tool_call["name"] == "add_to_order":
            # Add drink with modifiers to order
            modifiers = tool_call["args"].get("modifiers", [])
            modifier_str = ", ".join(modifiers) if modifiers else "no modifiers"

            order.append(f'{tool_call["args"]["drink"]} ({modifier_str})')
            response = "\n".join(order)

        elif tool_call["name"] == "confirm_order":
            # Display order to user for confirmation
            print("Your order:")
            if not order:
                print("  (no items)")

            for drink in order:
                print(f"  {drink}")

            response = input("Is this correct? ")

        elif tool_call["name"] == "get_order":
            # Return current order contents
            response = "\n".join(order) if order else "(no order)"

        elif tool_call["name"] == "clear_order":
            # Remove all items from order
            order.clear()
            response = "Order cleared"

        elif tool_call["name"] == "place_order":
            # Send order to kitchen
            order_text = "\n".join(order)
            print("Sending order to kitchen!")
            print(order_text)

            # Simulate order processing
            order_placed = True
            response = str(randint(1, 5))  # Random ETA in minutes

        else:
            raise NotImplementedError(f'Unknown tool call: {tool_call["name"]}')

        # Create tool response message
        outbound_msgs.append(
            ToolMessage(
                content=str(response),
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )

    return {"messages": outbound_msgs, "order": order, "finished": order_placed}

print("✅ Order processing node implemented")

✅ Order processing node implemented


This node handles all order-related operations, managing the order state and providing user interaction for order confirmation.

## Step 14: Enhanced Routing Logic

In [19]:
def maybe_route_to_tools(state: OrderState) -> str:
    """Enhanced routing between chat, tools, ordering, and end states."""
    if not (msgs := state.get("messages", [])):
        raise ValueError(f"No messages found when parsing state: {state}")

    msg = msgs[-1]

    # Check if order is completed
    if state.get("finished", False):
        return END

    # Route based on tool calls
    elif hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
        # Check if any tool calls are automated tools
        if any(tool["name"] in tool_node.tools_by_name.keys() for tool in msg.tool_calls):
            return "tools"  # Automated tools
        else:
            return "ordering"  # Manual order tools

    else:
        return "human"  # No tools, go to human interaction

print("✅ Enhanced routing logic implemented")

✅ Enhanced routing logic implemented


We implement sophisticated routing logic to handle different types of tool calls and application states.


## Step 15: Complete Graph Assembly

In [21]:
# Define tool sets
auto_tools = [get_menu]  # Automated tools
tool_node = ToolNode(auto_tools)

order_tools = [add_to_order, confirm_order, get_order, clear_order, place_order]  # Manual tools

# Bind all tools to the model
llm_with_tools = llm.bind_tools(auto_tools + order_tools)

# Update chatbot to use all tools
def chatbot_with_tools(state: OrderState) -> OrderState:
    """Final chatbot with all tool capabilities."""
    defaults = {"order": [], "finished": False}

    if state["messages"]:
        new_output = llm_with_tools.invoke([BARISTABOT_SYSINT] + state["messages"])
    else:
        new_output = AIMessage(content=WELCOME_MSG)

    return defaults | state | {"messages": [new_output]}

# Build complete graph
graph_builder = StateGraph(OrderState)

# Add all nodes
graph_builder.add_node("chatbot", chatbot_with_tools)
graph_builder.add_node("human", human_node)
graph_builder.add_node("tools", tool_node)
graph_builder.add_node("ordering", order_node)

# Define all routing - CORRECTIONS ICI
graph_builder.add_conditional_edges("chatbot", maybe_route_to_tools)  # ← avec S
graph_builder.add_conditional_edges("human", maybe_exit_human_node)   # ← avec S
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("ordering", "chatbot")
graph_builder.add_edge(START, "chatbot")

# Compile final graph
graph_with_order_tools = graph_builder.compile()

print("✅ Complete BaristaBot system assembled")

✅ Complete BaristaBot system assembled


We assemble the complete graph with all nodes and routing logic, creating a fully functional cafe ordering system.

## Step 16: Run Complete System

In [22]:
# Configure higher recursion limit for complex conversations
config = {"recursion_limit": 100}

print("🤖 Starting BaristaBot - Type 'q' to quit")
print("="*50)

# Initialize and run the complete system
try:
    state = graph_with_order_tools.invoke({"messages": []}, config)

    print("\n" + "="*50)
    print("Session completed!")
    print(f"Final order: {state.get('order', [])}")
    print(f"Order placed: {state.get('finished', False)}")

except KeyboardInterrupt:
    print("\nSession interrupted by user")
except Exception as e:
    print(f"An error occurred: {e}")

🤖 Starting BaristaBot - Type 'q' to quit
Model: Welcome to the BaristaBot cafe. Type `q` to quit. How may I serve you today?
User: cafe
Model: I can make you any of the following:  Espresso, Americano, Cold Brew, Latte, Cappuccino, Cortado, Macchiato, Mocha, Flat White, English Breakfast Tea, Green Tea, Earl Grey, Chai Latte, Matcha Latte, London Fog, Steamer, or Hot Chocolate.  I can also add any of the following modifiers:  Whole milk, 2% milk, Oat milk, Almond milk, 2% Lactose Free milk, single, double, triple, or quadruple espresso shots, decaf, hot or iced, vanilla sweetener, hazelnut sweetener, caramel sauce, chocolate sauce, sugar free vanilla sweetener, and any other reasonable special requests.  What would you like to order?

User: espresso
Model: Okay, one espresso.  Anything else for you?

User: breakfast
Model: I'm sorry, we don't serve breakfast.  We do have a variety of coffee and tea drinks, though.  Would you like to order something else from our menu?


Session interru